In [64]:
import pandas as pd
import numpy as np
df=pd.read_csv('D:/Automation code/LINE_LEVEL_DATAFRAME_FOR_PMHMD2.csv')
df.fillna(0)
# extract the characters and emotions from the data 

characters=[i.replace('_prop_text','') for i in df.columns if i.find('_prop_text')>-1]
emotions=['joy','sadness','disgust','anger','surprise','fear']
character_pair=list(df.iloc[:,list(df.columns).index('Words in paragraph'):list(df.columns).index('Line_proportion')].columns)[1:]
characters=[i.replace('_prop_text','') for i in df.columns if i.find('_prop_text')>-1]
col_names_to_show=[]
col_names_to_show.extend(('Date','Para no.','Line no.','Line length','Line_proportion','Para_proportion','Words in episode'))
col_names_to_show=col_names_to_show+emotions+characters+character_pair
total_chars_present=characters+character_pair
# create new emotion variables which stores the normalized results
new_emotions=[emotion+'new' for emotion in emotions]
# filter the df to get only the relevant columns 
df_1=df[col_names_to_show]

def create_emotion_variables():
    for i,emotion in enumerate(emotions):
        df_1[new_emotions[i]]=df_1[emotion].multiply(df_1['Para_proportion'],axis='index')

create_emotion_variables()
# create  a list for all distinct dates 
all_dates=list(df['Date'].unique())
all_dates_df=pd.DataFrame({'date':all_dates})

# create a temp df to append the information
master=pd.DataFrame(columns=emotions)

# create a temp df for each character to perform opertations
# run loop for each character 
for char_index,character in enumerate(total_chars_present):
    
    temp_df=df_1[df[total_chars_present[char_index]]>=1]

    dates_for_char_present=list(temp_df['Date'].unique())

    for new_emotion in new_emotions:
            #print new_emotion
            globals()[new_emotion]=list(temp_df.groupby(['Date'])[new_emotion].sum())

    # create a dictionary to store the values for each date 

    dates_values_emotions=dict((item[0],list(item[1:])) for item in zip(dates_for_char_present,joynew,sadnessnew,disgustnew,angernew,surprisenew,fearnew))

    # create the date-character leveldataframe and rename the first axis 

    char_level_emotion_date = pd.DataFrame(dates_values_emotions).transpose()
    char_level_emotion_date['char']=total_chars_present[char_index]
    char_level_emotion_date['date']=char_level_emotion_date.index

    # left join the 2 data frames so that if a date is not present for a character it shows NaN
    new=(pd.merge(all_dates_df,char_level_emotion_date,on='date',how='left'))
    new['char'].fillna(total_chars_present[char_index],inplace=True)
    #rename the column names joynew,sadnessnew,disgustnew,angernew,surprisenew,fearnew
    new.rename(columns={0:'joy',1:'sadness',2:'disgust',3:'anger',4:'surprise',5:'fear'},inplace=True)
    # append it to a temp_dataframe 
    master = master.append(new,ignore_index=True)
master.fillna(0,inplace=True)
master= pd.melt(master, id_vars=['char','date'], var_name=['emotion'])
#result.to_csv('Desktop/t.csv',index=False)
master.to_csv('Desktop/phmpd_t.csv',index=False)

C:\Users\User\Anaconda2\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
